In [1]:
import sys
sys.path.append('/Users/utkuozbudak/Desktop/Utku/TUM/Thesis/thesis')

from data.training_data_client import TrainingDataClient
from src.training.experiment_runner import ExperimentRunner
from src.training.experiment_runner_pod import ExperimentRunnerPOD
from src.training.dense import Dense
from src.training.linear import Linear
from src.training.deeponet import DeepONet
from src.training.pod_deeponet import DeepONetPOD
# sklearn pipeline
from sklearn.pipeline import Pipeline

### Read the data and split it into training and test sets

In [2]:
X_train, X_test, y_train, y_test, grid = TrainingDataClient.load_training_data(dataset="burgers", 
                                                                               test_size=0.2,
                                                                               verbose=True)

X_train shape: (12000, 256)
X_test shape: (3000, 256)
y_train shape: (12000, 256)
y_test shape: (3000, 256)
grid shape: (256, 1)


### DeepONet experimentation

In [3]:
results = ExperimentRunner.run(X_train, X_test, y_train, y_test, grid)

Current experiment:
n_modes:  64
Branch net config:  {'dense_layer': {'layer_width': 128, 'activation': 'tanh', 'parameter_sampler': 'tanh'}, 'linear_layer': {'regularization_scale': 0.01}}
Trunk net config:  {'dense_layer': {'layer_width': 128, 'activation': 'tanh', 'parameter_sampler': 'tanh'}, 'linear_layer': {'regularization_scale': 0.01}}
deeponet pod modes shape: (256, 64)
Iteration 1 | Relative L2 Loss: 0.7893071632801314
Iteration 2 | Relative L2 Loss: 0.10414893045117682
Iteration 3 | Relative L2 Loss: 0.10343283497968561
Iteration 4 | Relative L2 Loss: 0.10429060788630945
Iteration 5 | Relative L2 Loss: 0.10629682833973754
Iteration 6 | Relative L2 Loss: 0.09874315463121909
Iteration 7 | Relative L2 Loss: 0.1041466907653888
Iteration 8 | Relative L2 Loss: 0.10084909214518908
Iteration 9 | Relative L2 Loss: 0.1042834177404801
Iteration 10 | Relative L2 Loss: 0.10119354074789491
deeponet pod modes shape: (256, 64)
Iteration 1 | Relative L2 Loss: 0.7927183394689801
Iteration 2 |

### POD-DeepONet experimentation

In [3]:
pod_results = ExperimentRunnerPOD.run(X_train, X_test, y_train, y_test, grid)

Current experiment:
n_modes:  8
PODDeepONet Network config:  {'dense_layer': {'layer_width': 128, 'activation': 'tanh', 'parameter_sampler': 'tanh'}, 'linear_layer': {'regularization_scale': 1e-10}}
Dataset:  burgers


In [6]:
# DeepONet
branch_steps = [
    ("dense", Dense(layer_width=1024, 
                    activation="tanh",
                    parameter_sampler="tanh",
                    random_seed=42)),
    ("linear", Linear(regularization_scale=1e-6))
]
branch_net = Pipeline(branch_steps)

# Trunk net
trunk_steps = [
    ("dense", Dense(layer_width=1024, 
                    activation="tanh",
                    parameter_sampler="tanh",
                    random_seed=42)),
    ("linear", Linear(regularization_scale=1e-6))
]
trunk_net = Pipeline(trunk_steps)

model = DeepONet(branch_pipeline=branch_net, trunk_pipeline=trunk_net)
model.fit(X_train, y_train, grid)

deeponet pod modes shape: (256, 32)
Iteration 1 | Relative L2 Loss: 0.7951699523489734
Iteration 2 | Relative L2 Loss: 0.00411596727832319
Iteration 3 | Relative L2 Loss: 0.004115960125835005
No improvement for the last 1 iterations
Iteration 4 | Relative L2 Loss: 0.004115911190228134
No improvement for the last 2 iterations
Iteration 5 | Relative L2 Loss: 0.004115894872721704
No improvement for the last 3 iterations
Converged after 5 iterations | Loss: 0.004115894872721704


{'iteration_1_loss': 0.7951699523489734,
 'iteration_1_mse_loss': 0.0482856281338776,
 'iteration_2_loss': 0.00411596727832319,
 'iteration_2_mse_loss': 2.086683938745811e-06,
 'iteration_3_loss': 0.004115960125835005,
 'iteration_3_mse_loss': 2.086574350504287e-06,
 'iteration_4_loss': 0.004115911190228134,
 'iteration_4_mse_loss': 2.0864645900572314e-06,
 'iteration_5_loss': 0.004115894872721704,
 'iteration_5_mse_loss': 2.086439644826229e-06,
 'is_converged': True,
 'iterations_to_convergence': 5}

In [3]:
# DeepONetPOD
branch_steps = [
    ("dense", Dense(layer_width=1024, 
                    activation="tanh",
                    parameter_sampler="tanh",
                    random_seed=42)),
    ("linear", Linear(regularization_scale=1e-6))
]
branch_net = Pipeline(branch_steps)
pod_model = DeepONetPOD(pipeline=branch_net)
pod_model.fit(X_train, y_train)

pod-deeponet pod modes shape (256, 32)
y shape: (12000, 256)pod_mean shape: (256,)pod_modes shape: (256, 32)
goal shape: (12000, 32)


{'l2_mean_relative_loss': 0.004115873184758208,
 'mse_loss': 2.085355467618138e-06}